<a href="https://colab.research.google.com/github/Vjy14/golf-/blob/main/golf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


train_dir = '/content/drive/MyDrive/ct new/train'
test_dir = '/content/drive/MyDrive/ct new/test'

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2
)

In [2]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 252 images belonging to 2 classes.
Found 21 images belonging to 2 classes.


In [3]:
base_model_tf = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3),classes=2)

94765736/94765736 [==============================] - 1s 0us/step


In [4]:
base_model_tf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,AveragePooling2D,Dense,Flatten,ZeroPadding2D,BatchNormalization,Activation,Add,Input,Dropout,GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.initializers import glorot_uniform
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [6]:
base_model_tf.trainable=False

pt=Input(shape=(224,224,3))
func=tensorflow.cast(pt,tensorflow.float32)
x=preprocess_input(func) #This function used to zero-center each color channel wrt Imagenet dataset
model_resnet=base_model_tf(x,training=False)
model_resnet=GlobalAveragePooling2D()(model_resnet)
model_resnet=Dense(128,activation='relu')(model_resnet)
model_resnet=Dense(64,activation='relu')(model_resnet)
model_resnet=Dense(2,activation='softmax')(model_resnet)


model_main=Model(inputs=pt,outputs=model_resnet)
model_main.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.cast (TFOpLambda)        (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (  (None, 224, 224, 3)       0         
 SlicingOpLambda)                                                
                                                                 
 tf.nn.bias_add (TFOpLambda  (None, 224, 224, 3)       0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0     

In [7]:
model_main.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
print(train_generator.samples)
print(test_generator.samples)

252
21


In [10]:
model_main.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

Epoch 1/20
8/8 [==============================] - 10s 1s/step - loss: 0.6409 - accuracy: 0.6071 - val_loss: 0.6749 - val_accuracy: 0.4762
Epoch 2/20
8/8 [==============================] - 6s 758ms/step - loss: 0.6436 - accuracy: 0.5873 - val_loss: 0.6571 - val_accuracy: 0.6667
Epoch 3/20
8/8 [==============================] - 5s 615ms/step - loss: 0.6367 - accuracy: 0.6468 - val_loss: 0.6823 - val_accuracy: 0.5714
Epoch 4/20
8/8 [==============================] - 6s 749ms/step - loss: 0.6440 - accuracy: 0.5635 - val_loss: 0.7028 - val_accuracy: 0.5714
Epoch 5/20
8/8 [==============================] - 6s 693ms/step - loss: 0.6480 - accuracy: 0.6389 - val_loss: 0.6682 - val_accuracy: 0.5714
Epoch 6/20
8/8 [==============================] - 5s 616ms/step - loss: 0.6564 - accuracy: 0.6032 - val_loss: 0.7381 - val_accuracy: 0.4762
Epoch 7/20
8/8 [==============================] - 5s 599ms/step - loss: 0.6678 - accuracy: 0.5516 - val_loss: 0.7116 - val_accuracy: 0.5238
Epoch 8/20
8/8 [======

In [15]:
accuracy =model_main.evaluate(test_generator)
print(f'Test accuracy: {accuracy[1]*100:.2f}%')

1/1 [==============================] - 0s 444ms/step - loss: 0.6338 - accuracy: 0.6190
Test accuracy: 61.90%


In [16]:
model_main.save('golf.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
from tensorflow.keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('golf.h5')

test_image_path = '/content/drive/MyDrive/ct new/train/ct train/jason-pofahl-_Vuvc7-4ToA-unsplash_jpg.rf.9f8671127cef149f4deb7257ba2078dc.jpg'
img = image.load_img(test_image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0


predictions = model.predict(img_array)

class_labels = ['correct pose','wrong pose']
predicted_class = class_labels[np.argmax(predictions)]

print(f'The predicted class is: {predicted_class}')

1/1 [==============================] - 1s 869ms/step
The predicted class is: correct pose
